<a href="https://colab.research.google.com/github/maxrgnt/pythdc2-project2/blob/master/EDA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Panel Data
import pandas as pd
# System folders
import os
from pathlib import Path
# Visualization
import plotly.graph_objs as go
import seaborn as sns
%matplotlib inline

In [6]:
url = 'https://raw.githubusercontent.com/maxrgnt/pythdc2-project2/master/data/unemployment.csv'
bls = pd.read_csv(url)
print(bls.shape)
bls.sample(1)

(322, 5)


,State,Year,Abrv,UnemploymentRate,LaborRate
85,Arizona,2002,AZ,0.06495,0.659209


In [7]:
url = 'https://raw.githubusercontent.com/maxrgnt/pythdc2-project2/master/data/pctChangeGDP.csv'
bea = pd.read_csv(url)
print(bea.shape)
bea.sample(1)

(322, 4)


,State,Abrv,Year,Value
43,Arizona,AZ,1999,8.3


In [8]:
# read in the data
url='https://raw.githubusercontent.com/maxrgnt/pythdc2-project2/master/data/borderCrossing.csv'
bts = pd.read_csv(url)
print(bts.shape)
bts.sample(1)

(114588, 8)


,Abrv,State,Longitude,Latitude,Border,Year,Measure,Value
103980,ME,Maine,-67.78,46.45,US-Canada Border,1998,Train Passengers,0


In [0]:
def calcPctChange(df, col2calc):
  for y in df[col2calc].unique():
    print(y)
    blah = 1.0
    if y != df['Year'].min():
      t0 = df.loc[(df['Year'] == y-1) & (df['State'] == x),'Value'].tolist()[0]
      t1 = df.loc[(df['Year'] == y) & (df['State'] == x),'Value'].tolist()[0]
      df.loc[df['Year'] == y, 'pctChange'] = (1-t1/t0)*100
  print('hi')

In [0]:
# calcPctChange(df2, 'Year')
def listCol(df, col):
  years = list(df['Year'].unique())
  states = list(df['Abrv'].unique())
  for y in years:
    for s in states:
      print(y.astype(str),s)
      df.groupby([y.astype(str),s])[col].sum()

In [35]:
listCol(bea,'Value')

1996 AL


KeyError: ignored

In [0]:
for y in list(brdr['Year'].unique()):
  blah = 1.0
  if y != brdr['Year'].min():
    t0 = brdr.loc[brdr['Year'] == y-1,'immigrants'].tolist()[0]
    t1 = brdr.loc[brdr['Year'] == y,'immigrants'].tolist()[0]
    brdr.loc[brdr['Year'] == y, 'pct'] = (1-t1/t0)*100
brdr = brdr[brdr['Year'].astype(int)>brdr['Year'].min()]

Alaska
Arizona
California
Idaho
Maine
Michigan
Minnesota
Montana
New Mexico
New York
North Dakota
Ohio
Texas
Vermont
Washington


NameError: ignored